<a href="https://colab.research.google.com/github/CameronBeneteau/CamFinder/blob/main/PNL_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

class TradeSimulator:

    def __init__(self, ticker):

        self.ticker = ticker
        self.entry_price = None
        self.exit_price = None
        self.data = None

    def get_last_trading_day(self):

        today = datetime.now()
        offset = max(1,(today.weekday() + 6) % 7 -3)
        last_trading_day = today - timedelta(days=offset)
        return last_trading_day

    def getData(self):

        last_trading_day = self.get_last_trading_day()
        start_date = (last_trading_day - timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = (last_trading_day + timedelta(days=1)).strftime('%Y-%m-%d')
        self.data = yf.download(self.ticker, start=start_date, end=end_date, interval = '5m' )
        self.data = self.data.loc[last_trading_day.strftime('%Y-%m-%d')]

    def startTrade(self):

        self.entry_price = self.data['Open'].iloc[0]

    def exitTrade(self):

        for i, row in self.data.iterrows():

            current_price = row['Open']
            if current_price <= self.entry_price - (self.entry_price * 0.01) or current_price >= self.entry_price + (self.entry_price * 0.02):
                self.exit_price = current_price
                return

        self.exit_price = self.data['Close'].iloc[-1]

    def calculatePNL(self):

        return self.exit_price - self.entry_price

    def executeTrade(self):

        self.getData()
        self.startTrade()
        self.exitTrade()

        return {'Entry Price': self.entry_price, 'Exit Price': self.exit_price, 'PnL': self.calculatePNL()}

ticker = 'NVDA'
simulator = TradeSimulator(ticker)
result = simulator.executeTrade()
print(result)
# Random